In [1]:
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import os

# Define root directory
root = "."
sys.path.append(os.path.join(root, "..", "src"))
from default import CONFIGPATH

# Loading ChEMBL preprocessed data
print("Loading ChEMBL preprocessed data...")
ChEMBL = pd.read_csv(os.path.join(root, "..", "config", "chembl_processed", "activities_preprocessed.csv"), low_memory=False)
print(f"Original size: {len(ChEMBL)}")
print("Filtering out nan values...")
ChEMBL = ChEMBL[ChEMBL['value'].isna() == False].reset_index(drop=True)
print(f"Size after filtering nan values: {len(ChEMBL)}")

# Load directionality
direction = pd.read_csv(os.path.join(CONFIGPATH, "manual_curation", "activity_std_units_curated_manual_curation.csv"))
direction = direction[direction['manual_curation'].isna() == False]
direction = {(i,j): int(k) for i,j,k in zip(direction['activity_type'], direction['unit'], direction['manual_curation'])}

# List of pathogens to process
pathogens = ["Mycobacterium tuberculosis"]

Loading ChEMBL preprocessed data...
Original size: 24267312
Filtering out nan values...
Size after filtering nan values: 20911360


In [2]:
def get_assay_data(ChEMBL_, assay_id, activity_type, unit):
    if pd.isna(unit):
        assay_data = ChEMBL_[(ChEMBL_['assay_chembl_id'] == assay_id) & (ChEMBL_['activity_type'] == activity_type) & 
                                    (ChEMBL_['unit'].isna() == True) & (ChEMBL_['canonical_smiles'].isna() == False)]
    else:
        assay_data = ChEMBL_[(ChEMBL_['assay_chembl_id'] == assay_id) & (ChEMBL_['activity_type'] == activity_type) & 
                                    (ChEMBL_['unit'] == unit) & (ChEMBL_['canonical_smiles'].isna() == False)]
    return assay_data


def get_direction(activity_type, unit, direction):
    if (activity_type, unit) not in direction:
        raise ValueError(f"Activity type - unit not in direction dict: {activity_type} - {unit}. Curate manually.")
    else:
        return direction[(activity_type, unit)]


In [3]:
# For each pathogen
for pathogen in pathogens:
    
    # Get assays info
    pathogen_code = str(pathogen.split()[0][0] + pathogen.split()[1]).lower()
    print(f"\n\nFiltering data from pathogen: {pathogen_code}...")
    ChEMBL_ = ChEMBL[ChEMBL['target_organism'].str.contains(pathogen, case=False, na=False) | 
                    ChEMBL['assay_organism'].str.contains(pathogen, case=False, na=False)].reset_index(drop=True)
    
    # Load assays
    print(f"Number of activities for {pathogen}: {len(ChEMBL_)}")
    ASSAYS_INFO = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, 'assays.csv'))
    ASSAYS_INFO = ASSAYS_INFO[['assay_id', 'activity_type', 'unit', 'activities', 'cpds']].copy()
    print(f"Original number of assays: {len(ASSAYS_INFO)}")



Filtering data from pathogen: mtuberculosis...
Number of activities for Mycobacterium tuberculosis: 706206
Original number of assays: 11346


In [9]:
ASSAYS_INFO

,assay_id,activity_type,unit,activities,cpds
0,CHEMBL4649948,PERCENTEFFECT,%,93556,86590
1,CHEMBL4649949,PERCENTEFFECT,%,101516,86576
2,CHEMBL4649971,PERCENTEFFECT,%,68620,68614
3,CHEMBL4649972,PERCENTEFFECT,%,68617,68611
4,CHEMBL4649941,PERCENTEFFECT,%,67382,66942
...,...,...,...,...,...
11341,CHEMBL4434512,MIC,umol.L-1,1,1
11342,CHEMBL4434511,IC90,umol.L-1,1,1
11343,CHEMBL1675908,LOG10CFU,NaN,1,1
11344,CHEMBL1676664,MIC,umol.L-1,1,1


In [10]:
Counter(ChEMBL_['activity_comment'])

Counter({0: 699095, 1: 5720, -1: 1391})

In [7]:
# For each assay
for assay_id, activity_type, unit in tqdm(ASSAYS_INFO[['assay_id', 'activity_type', 'unit']].values):

    # Get assay data        
    assay_data = get_assay_data(ChEMBL_, assay_id, activity_type, unit)
        
    # Filter assay data
    assay_data = assay_data[['assay_chembl_id', 'target_chembl_id', 'compound_chembl_id', 'canonical_smiles', 'MW', 'pchembl', 
                             'activity_comment', 'standard_text', 'value', 'unit', 'activity_type', 'relation', 'pchembl_calculated']]
        
    # # Get direction
    # dir_ = get_direction(activity_type, unit, direction)

    # # Get values
    # values = assay_data[assay_data['relation'] == "="]['value'].tolist()
    # minimum, maximum = np.min(values), np.max(values)

    # # Adjust relations and values
    # # assay_data = adjust_relations_values(dir_, assay_data)
    # # ...
    # if dir_ == 1:

    #     # If dir is 1, <'s are 0's
    #     assay_data["value"] = [i if j != '<' else 0 for i,j in zip(assay_data['value'], assay_data['relation'])]

    # elif dir_ == -1:

    #     # If dir is -1, >'s are 0's
    #     assay_data["value"] = [i if j != '>' else 0 for i,j in zip(assay_data['value'], assay_data['relation'])]

        

    print(assay_id, " -- ", len(assay_data[assay_data['standard_text'] == 1]))

  0%|          | 2/11346 [00:00<20:35,  9.18it/s]

CHEMBL4649948  --  0
CHEMBL4649949  --  0


  0%|          | 4/11346 [00:00<20:05,  9.41it/s]

CHEMBL4649971  --  0
CHEMBL4649972  --  0


  0%|          | 6/11346 [00:00<20:57,  9.02it/s]

...

  3%|▎         | 292/11346 [00:28<18:37,  9.89it/s]

CHEMBL5374588  --  0
CHEMBL5374588  --  0
CHEMBL5262814  --  0


  3%|▎         | 294/11346 [00:28<18:09, 10.15it/s]

CHEMBL4703226  --  0
CHEMBL4831679  --  0


KeyboardInterrupt: 

In [ ]:
assays_with_annotations = dict(Counter(ChEMBL_[ChEMBL_['activity_comment'] != 0]['assay_chembl_id']))

In [24]:
for assay in sorted(assays_with_annotations):

    assay_data = ChEMBL_[ChEMBL_['assay_chembl_id'] == assay]
    actives = assay_data[assay_data['activity_comment'] == 1]
    undefined = assay_data[assay_data['activity_comment'] == 0]
    inactives = assay_data[assay_data['activity_comment'] == -1]

    break

In [27]:
actives

,activity_id,assay_id,assay_chembl_id,assay_type,assay_confidence_score,assay_organism,tid,target_type,target_organism,target_chembl_id,...,canonical_smiles,MW,pchembl,activity_comment,standard_text,value,unit,activity_type,relation,pchembl_calculated
22496,6540023,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,Cn1c(=O)c([N+](=O)[O-])c(-n2ccnc2)c2ccccc21,270.248,5.18,1,0,6.673,umol.L-1,AC50,=,5.175679
22497,6540024,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,CCCCC(O)(CCCC)C(=O)NN(C(=O)CCl)c1ccccc1,354.878,4.82,1,0,15.040,umol.L-1,AC50,=,4.822752
22498,6540025,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,COc1ccc(Nc2ccc(OC)cc2)cc1,229.279,NaN,1,0,121.500,umol.L-1,AC50,=,3.915424
22499,6540026,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,CCn1c(SCC(=O)Nc2nc(-c3ccc(C)cc3)cs2)nnc1-c1ccc...,451.577,5.46,1,0,3.460,umol.L-1,AC50,=,5.460924
22500,6540027,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,O=C(Oc1ccc2ccccc2c1)c1ccco1,238.242,4.74,1,0,18.310,umol.L-1,AC50,=,4.737312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24642,6596119,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,O=S(=O)(CCSc1nc2ccccc2s1)c1nc2ccccc2s1,392.552,5.64,1,0,2.291,umol.L-1,AC50,=,5.639975
24645,6596122,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,c1coc(-c2nnc(-c3ccco3)nn2)c1,214.184,5.24,1,0,5.786,umol.L-1,AC50,=,5.237622
24646,6596123,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,COc1ccc(CC2=CC(=O)c3ccccc3C2=O)cc1,278.307,5.13,1,0,7.490,umol.L-1,AC50,=,5.125518
24647,6596124,752386,CHEMBL1794324,F,8,NaN,103988,SINGLE PROTEIN,Mycobacterium tuberculosis,CHEMBL1741192,...,COc1ccc(N=C(S)NC(NC(=O)CCl)C(Cl)(Cl)Cl)c([N+](...,450.131,4.54,1,0,28.530,umol.L-1,AC50,=,4.544698


In [ ]:
plt.hist(assay_data['value'].tolist(), bins=[i for i in range(-200, 100, 5)])
plt.show()

In [ ]:
# Binarization

# 1. Get direction
# if dir_ == +1 ==> > are =, < are min
# if dir_ == -1 ==> < are =, > are max

# 2. 